In [ ]:
#This code is to run the pandas on GPU
# Note : some times error may occur because of change in versions

#!pip install cudf-cu11==23.10 rmm-cu11==23.10 --extra-index-url=https://pypi.nvidia.com
!pip install cudf-cu12 --extra-index-url=https://pypi.nvidia.com

import cudf


In [1]:
#Importing the required packages
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

import warnings
from pandas.errors import SettingWithCopyWarning

import json
import nltk
from nltk.tokenize import word_tokenize
import re
import math
import os
nltk.download('punkt')

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Connecting to the drive
from google.colab import drive
drive.mount('/content/drive',force_remount = True)

Mounted at /content/drive


In [3]:
# Code to change the directory path where the input file is present
os.chdir('/content/drive/MyDrive/')


In [ ]:
# Function to preprocess the text
def preproce(txt):
    # Substitute punctuation marks with spaces
    txt = re.sub(r'''      # Start raw string block
               \W+       # Accept one or more non-word characters
               \s*       # plus zero or more whitespace characters,
               ''',      # Close string block
               ' ',      # and replace it with a single space
               txt,
               flags=re.VERBOSE)

    # lower the txt
    txt = txt.lower()

    # tockenize the text
    txt = word_tokenize(txt)

    return txt



In [ ]:
# Read the corpus data
json_file = "2022-11-30-papers.jsonl" # write input file name
withoutPreProcessing = []
withPreProcessing = []
totalToken = 0
with open(json_file, 'r') as f:
    try:
        for i, line in enumerate(f):
            data = json.loads(line) # data is a Python dictionary
            #print(data)
            title = data["title"]
            abstract = data["abstract"]

            # preprocess title and abstract and store
            title_tokens = preproce(title)
            abstract_tokens = preproce(abstract)
            #Storing the tokens in the dictornary
            withPreProcessing.append({
                "title_tokens":list(title_tokens),
                "abstract:tokens":list(abstract_tokens),
                })


    except Exception as e:
        print(e)


In [ ]:
# Code to gather the title & abstract tokens of the corpus data
Title = [itm  for _ in withPreProcessing for itm in _['title_tokens']]
Abstract = [itm  for _ in withPreProcessing for itm in _['abstract:tokens']]
# Store both title & abstract tokens
total_tokens = Title + Abstract
print(f"Total number of Titles and Abstract present {len(total_tokens)}")


In [ ]:
# Uncomment below lines when working on huge gold standard dataset
# ------------------------------------------------------
# code to save the total tokens in pkl format
#import pickle
#with open('Token_list.pkl', 'wb') as f:
#  pickle.dump(total_tokens, f)
# ------------------------------------------------------
# Code to read the total token from a pickle file
import pickle
with open('Token_list.pkl', 'rb') as f:
  total_tokens = pickle.load(f)


In [44]:

#file_input_path = 'depstructures3(input).xlsx'
file_input_path ='Gold Standard(new) Output.xlsx'
sheetname1= "Triplets"
sheetname2 = "MWTs"

# read the data
data = pd.read_excel(file_input_path, sheet_name=sheetname2)
#data.MWT = data.MWT.astype('str')

In [ ]:

def C_value(mwt,data):
  '''
  This performs C value calculation

  param mwt : Multi word term for which we are going to perform{ data['MWT']}
  data : Data frame object
  '''
  index = data[data['MWT']== mwt].index.values[0]
  mod_a = len(mwt.split(' '))
  f_a = data['Absolute frequency '][index]

  T_a = data[data["MWT"].str.contains(mwt,na=False)]
  # Excluding the mwt also
  T_a = T_a[T_a.MWT!=mwt]

  if T_a.shape[0] == 0 :
    return math.log(mod_a,2) * f_a

  else :

    P_T_a = T_a.shape[0]
    summation_f_b = T_a['Absolute frequency '].sum()

    return math.log(mod_a,2) * (f_a - (summation_f_b/P_T_a))


In [ ]:
# Code to run the pandas on GPU/CPU
%load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [53]:
data.head(4)

,MWT,Dependency pairs,Absolute frequency,Relative frequency,C-value,Embedding vector,Unnamed: 6,Unnamed: 7,Absolute frequency .1,Relative frequency .1,C-value .1
0,smart city unmanned aerial,"unmanned aerial, smart city, city aerial",2,1.966488e-08,1.000000,"[1.109797477722168, 0.6286604404449463, -1.278...",NaN,Minimal value,1.000000,9.832438e-09,1.000000
1,classical cellular automaton,"cellular automaton, classical automaton",1,9.832438e-09,1.584963,"[0.40245428681373596, 0.6819993853569031, 0.07...",NaN,Maximal value,2344.000000,2.304723e-05,3695.604231
2,cyclic cellular automaton,"cellular automaton, cyclic automaton",1,9.832438e-09,1.584963,"[0.25442272424697876, 0.29363080859184265, 0.2...",NaN,Mean value,11.164415,1.097734e-07,18.340257
3,urban cellular automaton,"cellular automaton, urban automaton",1,9.832438e-09,1.584963,"[-0.16562871634960175, 0.791401207447052, -0.3...",NaN,NaN,NaN,NaN,NaN


In [6]:
print(pd)

<module 'pandas' from '/usr/local/lib/python3.10/dist-packages/pandas/__init__.py'>


In [ ]:
# Convert the tokens in the list to an entire string text. This is performed to use regex
text = ' '.join(total_tokens)

NameError: name 'total_tokens' is not defined

In [ ]:
# Calculates the Absolute frequency
def count_multiword_occurrences(text, multiword):
    #pattern = re.compile(r'\b' + re.escape(multiword) + r'\b')
    pattern = multiword
    matches = re.findall(pattern, text)
    return len(matches)


In [ ]:
# Calculates the Absolute frequency
data['Absolute frequency '] = data['MWT'].apply(lambda x : count_multiword_occurrences(text,x))

In [ ]:
data['Absolute frequency ']

0          2
1          1
2          1
3          1
4          1
        ... 
5178     971
5179    1343
5180    1083
5181    1382
5182    2344
Name: Absolute frequency , Length: 5183, dtype: int64

In [ ]:
data.to_excel("temp_abs1.xlsx", sheet_name=sheetname2,index=False)

In [ ]:
# Calculates the relative frequency
no_tokens = 101704177
data['Relative frequency '] = data['Absolute frequency '].apply(lambda x : x/no_tokens)


In [ ]:
# Calculate the C value
data['C-value '] = data['MWT'].apply(lambda x : C_value(x,data))

In [6]:
#Check the top 5 rows
data.head()

,MWT,Dependency pairs,Absolute frequency,Relative frequency,C-value,Embedding vector,Unnamed: 6,Unnamed: 7,Absolute frequency .1,Relative frequency .1,C-value .1
0,smart city unmanned aerial,NaN,NaN,NaN,NaN,"[1.109797477722168, 0.6286604404449463, -1.278...",NaN,Minimal value,NaN,NaN,NaN
1,classical cellular automaton,NaN,NaN,NaN,NaN,"[0.40245428681373596, 0.6819993853569031, 0.07...",NaN,Maximal value,NaN,NaN,NaN
2,cyclic cellular automaton,NaN,NaN,NaN,NaN,"[0.25442272424697876, 0.29363080859184265, 0.2...",NaN,Mean value,NaN,NaN,NaN
3,urban cellular automaton,NaN,NaN,NaN,NaN,"[-0.16562871634960175, 0.791401207447052, -0.3...",NaN,NaN,NaN,NaN,NaN
4,fuzzy covariance matrix,NaN,NaN,NaN,NaN,"[0.10261454433202744, 0.5573097467422485, 0.54...",NaN,NaN,NaN,NaN,NaN


In [ ]:
data.columns

In [66]:
data['Absolute frequency .1'][0] = min(data['Absolute frequency '])
data['Absolute frequency .1'][1] = max(data['Absolute frequency '])
data['Absolute frequency .1'][2] = data['Absolute frequency '].mean()

data['Relative frequency .1'][0] = min(data['Relative frequency '])
data['Relative frequency .1'][1] = max(data['Relative frequency '])
data['Relative frequency .1'][2] = data['Relative frequency '].mean()


data['C-value .1'][0] = min(data['C-value '])
data['C-value .1'][1] = max(data['C-value '])
data['C-value .1'][2] = data['C-value '].mean()





In [9]:
# Scibert Mean Pooling on MWT

model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [27]:
# Create the Embedding vetor
data['Embedding vector'] = data.MWT.apply(lambda x:
    torch.mean(model(**tokenizer(str(x), return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).squeeze().detach().numpy().tolist()
)


In [48]:
#read the excel file
data_1 = pd.read_excel(file_input_path, sheet_name = sheetname1)
data_1.head(10)

,Hypernym,Anchor term,Closely related term,Distantly related term,Cosine similarity of closely related terms (cosim1),Cosine similarity of distantly related terms (cosim2),difference (cosim1-cosim2),Unnamed: 7,Unnamed: 8,Cosine similarity of closely related terms (cosim1).1,Cosine similarity of distantly related terms (cosim2).1,
0,access control,medium access control,network access control,temporal access control,0.922089,0.868728,0.053361,NaN,Minimal value,0.576473,0.54571,NaN
1,access control,trend analysis dynamic access control,authentication identity management access control,manageable access control,0.810207,0.840561,-0.030354,NaN,Maximal value,0.989592,0.97388,NaN
2,access control,mutual authentication access control,user authentication access control,service access control,0.941694,0.883724,0.057970,NaN,Mean value,0.892154,0.85775,NaN
3,access control,low cost private office access control,real time automatic access control,cryptographic protocol analysis access control,0.795672,0.730794,0.064878,NaN,NaN,NaN,NaN,NaN
4,access control,dynamic access control,hierarchical access control,mandatory access control,0.948542,0.906620,0.041922,one-tailed (greater),t-score,NaN,p-value,NaN
5,access control,multidimensional medium access control,single channel cognitive medium access control,energy efficient medium access control,0.879718,0.904971,-0.025254,one-tailed (greater),w-score,NaN,p-value,NaN
6,access control,flexible access control,powerful access control,indirect access control,0.934493,0.909177,0.025316,two-tailed,t-score,NaN,p-value,NaN
7,access control,conventional network access control,specific medium access control,visitor access control,0.900969,0.823248,0.077722,two-tailed,w-score,398706.000000,p-value,7.245031e-131
8,access control,robust access control,reliable access control,modular access control,0.910952,0.897027,0.013924,NaN,Shapiro-Wilk score,0.977155,p-value,1.366462e-17
9,access control,personal identification access control,web service access control,anonymous biometric access control,0.826600,0.825904,0.000696,NaN,NaN,NaN,NaN,NaN


In [37]:
# function to calculate cosine similarity between two vectors
import ast
def cosine_sim(vec1, vec2):

  #vec1 = ast.literal_eval(vec1)
  #vec2 = ast.literal_eval(vec2)
  return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

#These variables are used for storing the embedding vectors
data_1['Distantly related term vector']  = None
data_1['Closely related term vector'] = None
data_1['Anchor term vector'] = None


In [38]:

# Code to create the vectors
for i in range(data_1.shape[0]):

  #print(i)
  try:
    close_vector = data['Embedding vector'][data[data['MWT']== data_1['Closely related term '][i]].index.values[0]]
    data_1['Closely related term vector'][i] = close_vector
  except:
    data_1['Closely related term vector'][i] = torch.mean(model(**tokenizer(str(data_1['Distantly related term '][i]), return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).squeeze().detach().numpy().tolist()

  try :
    distant_vector = data['Embedding vector'][data[data['MWT']== data_1['Distantly related term '][i]].index.values[0]]
    data_1['Distantly related term vector'][i] = distant_vector
  except:
    data_1['Distantly related term vector'][i] = torch.mean(model(**tokenizer(str(data_1['Distantly related term '][i]), return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).squeeze().detach().numpy().tolist()

  try:
    anchor_vector = data['Embedding vector'][data[data['MWT']== data_1['Anchor term '][i]].index.values[0]]
    data_1['Anchor term vector'][i] = anchor_vector
  except:
    data_1['Anchor term vector'][i] = torch.mean(model(**tokenizer(str(data_1['Distantly related term '][i]), return_tensors='pt', padding=True, truncation=True)).last_hidden_state, dim=1).squeeze().detach().numpy().tolist()

  # calculate the cosine similarity

  data_1['Cosine similarity of closely related terms (cosim1)'][i] = cosine_sim(data_1['Anchor term vector'][i], data_1['Closely related term vector'][i])
  data_1['Cosine similarity of distantly related terms (cosim2)'][i] = cosine_sim(data_1['Anchor term vector'][i],data_1['Distantly related term vector'][i])

In [39]:
data_1['difference (cosim1-cosim2)'] = data_1['Cosine similarity of closely related terms (cosim1)']- data_1['Cosine similarity of distantly related terms (cosim2)']


In [40]:
# Calulate the Min,Max & Mean
data_1['Cosine similarity of closely related terms (cosim1).1'][0] = min(data_1['Cosine similarity of closely related terms (cosim1)'])
data_1['Cosine similarity of closely related terms (cosim1).1'][1] = max(data_1['Cosine similarity of closely related terms (cosim1)'])
data_1[ 'Cosine similarity of closely related terms (cosim1).1'][2] = data_1['Cosine similarity of closely related terms (cosim1)'].mean()

data_1['Cosine similarity of distantly related terms (cosim2).1'][0] = min(data_1['Cosine similarity of distantly related terms (cosim2)'])
data_1['Cosine similarity of distantly related terms (cosim2).1'][1] = max(data_1['Cosine similarity of distantly related terms (cosim2)'])
data_1[ 'Cosine similarity of distantly related terms (cosim2).1'][2] = data_1['Cosine similarity of distantly related terms (cosim2)'].mean()



In [41]:
# Drop the created vector columns
data_1 = data_1.drop(columns=['Distantly related term vector', 'Closely related term vector','Anchor term vector'])

In [51]:
import scipy.stats as stats
from scipy.stats import shapiro

# Calculating the T-test and wilcoxon Test
# Note : According the given schema the value is stored at this column
data_1[' '] = None

data_1['Cosine similarity of closely related terms (cosim1).1'][8] = shapiro(data_1['difference (cosim1-cosim2)'])[0]
data_1[' '][8] = shapiro(data_1['difference (cosim1-cosim2)'])[1]
pvalue = shapiro(data_1['difference (cosim1-cosim2)'])[1]

if pvalue >= 0.05:
  #two tailed t-test
  data_1['Cosine similarity of closely related terms (cosim1).1'][6] = stats.ttest_rel(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'],alternative='greater')[0]
  data_1[' '][6] = stats.ttest_rel(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'],alternative='greater')[1]

  #one tailed t test

  data_1['Cosine similarity of closely related terms (cosim1).1'][4] = stats.ttest_rel(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'])[0]
  data_1[' '][4] = stats.ttest_rel(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'])[1]




if pvalue < 0.05:
  #two tailed wilcoxon test
  data_1['Cosine similarity of closely related terms (cosim1).1'][7] = stats.wilcoxon(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'])[0]
  data_1[' '][7] = stats.wilcoxon(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'])[1]

  #one tailed wilcoxon test
  data_1['Cosine similarity of closely related terms (cosim1).1'][5] = stats.wilcoxon(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'],alternative='greater')[0]
  data_1[' '][5] = stats.wilcoxon(data_1['Cosine similarity of closely related terms (cosim1)'],data_1['Cosine similarity of distantly related terms (cosim2)'],alternative='greater')[1]


In [54]:
# Saving the dataframes to Excel file
with pd.ExcelWriter('Gold Standard(new) Output.xlsx') as writer:
  data_1.to_excel(writer, sheet_name=sheetname1,index=False)
  data.to_excel(writer, sheet_name=sheetname2,index=False)



